In [196]:
import pandas as pd 
import requests
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.shared import OxmlElement,qn 

import time
import hmac
import hashlib
import requests
from urllib.parse import urlencode
import pandas as pd 
import re
import json



class Signature:
    def __init__(self,KEY,SECRET,BASE_URL):
        self.KEY = KEY
        self.SECRET = SECRET
        self.BASE_URL = BASE_URL
        
    def hashing(self,query_string):
        query_string = query_string
        return hmac.new(
            self.SECRET.encode("utf-8"), query_string.encode("utf-8"), hashlib.sha256
        ).hexdigest()
    
    def get_timestamp(self):
        return int(time.time()*1000)
    
    def dispatch_request(self,apply_method):
        session = requests.Session()
        session.headers.update(
            {"Content-Type": "application/json;charset=utf-8", "X-MBX-APIKEY": self.KEY}
        )
        methods = {
            "GET": session.get,
            "DELETE": session.delete,
            "PUT": session.put,
            "POST": session.post,
        }
        if apply_method == list(methods.keys())[0]:
            return methods.get('GET')
        elif apply_method == list(methods.keys())[1]:
            return methods.get('DELETE')
        elif apply_method == list(methods.keys())[2]:
            return methods.get('PUT')   
        elif apply_method == list(methods.keys())[3]:
            return methods.get('POST')   
    
    def send_signed_request(self, apply_method,url_path, payload={}):
        
        try:
            query_string = urlencode(payload)
            # replace single quote to double quote
            query_string = query_string.replace("%27", "%22")
            if query_string:
                query_string = "{}&timestamp={}".format(query_string, self.get_timestamp())
            else:
                query_string = "timestamp={}".format(self.get_timestamp())

            url = (
                self.BASE_URL + url_path + "?" + query_string + "&signature=" + self.hashing(query_string)
            )
            print("{} {}".format(apply_method, url))
            params = {"url": url, "params": {}}
            response = self.dispatch_request(apply_method)(**params)
            return response.json()
        except Exception as e:
            print(e)
            print(self.dispatch_request(apply_method)(**params).text)

symbol = 'NEARUSDT'

api_key = 'oPptPTR7isZJazQD8haMcHsQSXyyp4FUN1HpR5iejo5ZKkd1pE3ZE0ReFhR8mmqk'
api_secret = 'iusAwWonkj6TfXhmrDNrwwhSRqtdfQUVeG8erh9RbuqYsOKnjmIteAmkgsoWdZPl'
url = 'https://fapi.binance.com'
sign = Signature(KEY=api_key,SECRET=api_secret,BASE_URL=url)
resp = sign.send_signed_request('GET','/fapi/v1/leverageBracket',{'symbol':symbol})

orig_brackets = pd.DataFrame(resp[0]['brackets'])
orig_brackets
new_brackets = pd.read_csv('brackets.csv')
new_brackets.head(10)




GET https://fapi.binance.com/fapi/v1/leverageBracket?symbol=NEARUSDT&timestamp=1727868639970&signature=cbb372f7a39bd0f7bf6bde3aed62d140f8037087399af816de8c4158ad0a559f


,symbol,tier,floor,cap,mmr,leverage,maintenance amount
0,NEARUSDT,1,0,"10,000",1.00%,75,0
1,NEARUSDT,2,"10,000","100,000",1.50%,50,50
2,NEARUSDT,3,"100,000","500,000",2.00%,25,550
3,NEARUSDT,4,"500,000","1,000,000",2.50%,20,"3,050"
4,NEARUSDT,5,"1,000,000","5,000,000",5.00%,10,"28,050"
5,NEARUSDT,6,"5,000,000","10,000,000",10.00%,5,"278,050"
6,NEARUSDT,7,"10,000,000","12,500,000",12.50%,4,"528,050"
7,NEARUSDT,8,"12,500,000","25,000,000",25.00%,2,"2,090,550"
8,NEARUSDT,9,"25,000,000","50,000,000",50.00%,1,"8,340,550"


In [197]:
doc = Document()

table = doc.add_table(rows=2,cols=6)
table.style = 'Table Grid' # 设置表格样式

# 写入第一行表头信息
first_header = table.rows[0].cells
first_header[0].text = 'Previous Leverage and Margin Tiers'
first_header[1].text = ''
first_header[2].text = ''
first_header[3].text = 'New Leverage and Margin Tiers'
first_header[4].text = ''
first_header[5].text = ''

for cell in first_header:
    for paragraph in cell.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = paragraph.runs[0]
        run.bold = True 

first_header[0].merge(first_header[1]).merge(first_header[2])
first_header[3].merge(first_header[4]).merge(first_header[5])

# 写入第二行表头信息

second_header = table.rows[1].cells
second_header[0].text = 'Leverage Before Change'
second_header[1].text = 'Position Before Change (Notional Value in USDT)'
second_header[2].text = 'Maintenance Margin Rate Before Change'
second_header[3].text = 'Leverage After Change'
second_header[4].text = 'Position After Change (Notional Value in USDT)'
second_header[5].text = 'Maintenance Margin Rate After Change'

for cell in second_header:
    for paragraph in cell.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = paragraph.runs[0]
        run.bold = True 

orig_brackets = orig_brackets[['initialLeverage','notionalCap','maintMarginRatio']]
orig_brackets['last_leverage']  = orig_brackets['initialLeverage'].shift(-1).fillna(1)+1
orig_brackets['levearage_str'] = orig_brackets['last_leverage'].astype(int).astype('str') + ' - '+orig_brackets['initialLeverage'].astype('str')+'x'
orig_brackets['check'] = (orig_brackets['initialLeverage'] - orig_brackets['last_leverage'])<=0
orig_brackets['levearage_str']  = orig_brackets.apply(lambda x: x if x['check']==False else x['levearage_str'].split('-')[1],axis=1)['levearage_str']
orig_brackets['position_notional_str']  =orig_brackets['notionalCap'].shift().fillna(0).astype(int).apply(lambda x:f'{x:,}')+' < Position ≤ ' + orig_brackets['notionalCap'].apply(lambda x:f'{x:,}')
orig_brackets['mmr_str']  = orig_brackets['maintMarginRatio'].apply(lambda x:str(round(x*100,2))+'0%')
orig_brackets= orig_brackets[['levearage_str','position_notional_str','mmr_str']]

orig_brackets = orig_brackets.iloc[:-2]

new_brackets = new_brackets[['leverage','cap','mmr']]
new_brackets.columns = ['initialLeverage','notionalCap','maintMarginRatio']
new_brackets['last_leverage']  = new_brackets['initialLeverage'].shift(-1).fillna(1)+1
new_brackets['levearage_str'] = new_brackets['last_leverage'].astype(int).astype('str') + ' - '+new_brackets['initialLeverage'].astype('str')+'x'
new_brackets['check'] = (new_brackets['initialLeverage'] - new_brackets['last_leverage'])<=0
new_brackets['levearage_str']  = new_brackets.apply(lambda x: x if x['check']==False else x['levearage_str'].split('-')[1],axis=1)['levearage_str']
new_brackets['position_notional_str']  = new_brackets['notionalCap'].shift().fillna('0')+' < Position ≤ ' +new_brackets['notionalCap']
new_brackets['mmr_str'] = new_brackets['maintMarginRatio']
new_brackets = new_brackets[['levearage_str','position_notional_str','mmr_str']]

df = pd.merge(orig_brackets,new_brackets,left_index=True, right_index=True,how='right',suffixes=['_before','_after'])
# print(df)

for index, row in df.iterrows():
    table_row = table.add_row().cells 
    # print(row.iloc[0:3].isnull())
    # break 

    # 检查前三个单元格是否位None，若为None则合并
    if row.iloc[0:3].isnull().all():
        merge_cell = table_row[0].merge(table_row[1]).merge(table_row[2])
        merge_cell.text = ''
    else:
        #否则，分别写入前三个单元格 
        for i in range(3):
            table_row[i].text = str(row[i]) if row[i] is not None else 'N/A'
    
    # 填写第四至第六列
    table_row[3].text = str(row[3])
    table_row[4].text = str(row[4])
    table_row[5].text = str(row[5])

    # 如果第二列和第5列数据不一样，给第五列字体加粗
    if str(row[1]) != str(row[4]):
        run5 = table_row[4].paragraphs[0].runs[0]  # 获取第5列的第一个run 
        run5.bold = True # 加粗
        run6 = table_row[5].paragraphs[0].runs[0]  # 获取第5列的第一个run 
        run6.bold = True # 加粗


doc.save('test_result.docx')


In [202]:
from docx import Document

def replace_placeholder(doc, placeholders):
    """
    读取Word文档并替换占位符
    :param doc: Document 对象
    :param placeholders: 占位符字典 {占位符: 替换内容}
    """
    for paragraph in doc.paragraphs:
        for placeholder, replacement in placeholders.items():
            if placeholder in paragraph.text:
                # 替换段落中的文本
                paragraph.text = paragraph.text.replace(placeholder, replacement)

    # 处理表格中的占位符
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for placeholder, replacement in placeholders.items():
                        if placeholder in paragraph.text:
                            # 替换表格中的文本
                            paragraph.text = paragraph.text.replace(placeholder, replacement)

def main():
    # 读取已有的Word文档
    template_path = 'bracket_anns_template.docx'  # 样式文档路径
    doc = Document(template_path)

    # 定义占位符和它们的替换内容
    placeholders = {
        '[date]': '2024-09-30',
        '[time]': '2024-09-30 10:30',
        '[symbols]': 'NEARUSDT',
        '[symbol]':'',
        '[brackets info]':table_row,
        '[cn_date]':'',
        '[cn_time]':'',
        '[cn_symbols]':''

    }

    # 替换占位符
    replace_placeholder(doc, placeholders)

    output_path = 'amended_anns.docx'
    doc.save(output_path)

main()

In [194]:
import pandas as pd 

data = pd.read_clipboard().dropna()

data.head()
data['futures_user_id'] = data['futures_user_id'].astype(int).astype(str)

In [195]:
new_df = pd.DataFrame({'fuids':data.groupby(data.index // 90)['futures_user_id'].apply(lambda x: '\n'.join(x))})

new_df.to_csv('uids.csv')

In [182]:
tf = []
new_list = []
user_list = [str(int(x)) for x in data['futures_user_id'].unique()]
for i in range(0,len(user_list),90):
    for z in range(len(user_list)):
        if i%90==0:
            continue 
        else:
            
            new_str = ','.join(z)
            new_list.append(new_str)
        tf.append(new_list)




In [183]:
new_list

[]

In [67]:
orig_brackets = orig_brackets[['initialLeverage','notionalCap','maintMarginRatio']]
orig_brackets['last_leverage']  = orig_brackets['initialLeverage'].shift(-1).fillna(1)+1
orig_brackets['levearage_str'] = orig_brackets['last_leverage'].astype(int).astype('str') + ' - '+orig_brackets['initialLeverage'].astype('str')+'x'
orig_brackets['position_notional_str']  =orig_brackets['notionalCap'].shift().fillna(0).astype(int).apply(lambda x:f'{x:,}')+' < Position ≤' + orig_brackets['notionalCap'].apply(lambda x:f'{x:,}')
orig_brackets['mmr_str']  = orig_brackets['maintMarginRatio'].apply(lambda x:str(round(x*100,2))+'0%')
orig_brackets= orig_brackets[['levearage_str','position_notional_str','mmr_str']]

new_brackets
orig_brackets






KeyError: "None of [Index(['initialLeverage', 'notionalCap', 'maintMarginRatio'], dtype='object')] are in the [columns]"